# MLOps workshop with Amazon SageMaker

## Module 03 (**optional**): Automate the whole dataset preparation and model training pipeline using Low-code Experience for SageMaker Pipelines.

We're introducing a low-code experience for data scientists to convert the Machine Learning (ML) development code into repeatable and reusable workflow steps of Amazon SageMaker Pipelines.

This notebook shows the example of orchestrating jobs for model building and batch inference using low-code experience for SageMaker Pipelines, utilizing `@step` decorator. We build an automated model building pipeline for a house prices prediction, based on the well-known California housing dataset with a simple regression model in Tensorflow 2

We will use the same dataset and model inroduced in [Module 02: Transform the data and train a model using SageMaker managed training job](../02_manual_sagemaker_process_train/02_manual_sagemaker_process_train.ipynb) notebook. 

**Note** this notebook can only run on `Base Python 3.0` Kernel. 

## Install the dependencies

We will create a `requirements.txt` file that will be used in this notebook, and in the pre-processing, training and evaluation jobs as part of the pipeline. 

In [ ]:
%%writefile requirements.txt

pandas==2.1.4
scikit-learn==1.3.2
tensorflow==2.15.0
sagemaker>=2.203.0,<3

Now we will install the dependencies on the notebook

In [ ]:
%pip install -r ./requirements.txt -q

## Setup Configuration file path

We are setting the directory in which the `config.yaml` file resides so that step decorator can make use of the settings.

You can see we use default `ml.m5.large` for the compute to be run with the `@step` decorator. Also, note `requirements.txt` will be installed as default.  

In [ ]:
%%writefile config.yaml

SchemaVersion: '1.0'
SageMaker:
  PythonSDK:
    Modules:
      RemoteFunction:
        # role arn is not required if in SageMaker Notebook instance or SageMaker Studio
        # Uncomment the following line and replace with the right execution role if in a local IDE
        # RoleArn: <replace the role arn here>
        InstanceType: ml.m5.large
        Dependencies: ./requirements.txt
        IncludeLocalWorkDir: true
        CustomFileFilter:
          IgnoreNamePatterns: # files or directories to ignore
          - "*.ipynb" # all notebook files

We can use configuration file `config.yaml` to set default values of the infrastructure such as instance type, and dependencies to run the pipeline. We use environment variable "SAGEMAKER_USER_CONFIG_OVERRIDE" to set the path to configuration file.

In [ ]:
import os
import boto3

# Set path to config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

## Dataset

First we'll load the California Housing dataset, save the raw feature data and use it in the step as part of the Low-code Experience for SageMaker Pipeline.  We'll also save the labels for training and testing.
    
More info on the dataset:

This dataset was obtained from the StatLib repository. http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).

In [ ]:
import boto3
import json
import sagemaker
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sagemaker.workflow.function_step import step
from sagemaker.workflow.parameters import ParameterString

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name

In [ ]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/california_housing/cal_housing.tgz .

In [ ]:
!tar -zxf cal_housing.tgz 2>/dev/null

In [ ]:
columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
    "medianHouseValue",
]
df = pd.read_csv("CaliforniaHousing/cal_housing.data", names=columns, header=None)

In [ ]:
columns_to_normalize = [
    'medianIncome', 'housingMedianAge', 'totalRooms', 
    'totalBedrooms', 'population', 'households', 'medianHouseValue'
]

for column in columns_to_normalize:
    df[column] = np.log1p(df[column])

In [ ]:
X = df.drop("medianHouseValue", axis=1)
Y = df["medianHouseValue"].copy()

In [ ]:
print("Features:", list(X.columns))
print("Dataset shape:", X.shape)
print("Dataset Type:", type(X))
print("Label set shape:", Y.shape)
print("Label set Type:", type(X))

In [ ]:
# We partition the dataset into 2/3 training and 1/3 test set.
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

## Define variables and pipeline parameters

In [61]:
pipeline_name = "tf-2-basic-lyft-and-shift-pipeline"

We will define a parameterized `instance_type`, so we can override the default `ml.m5.large` defined in `config.yaml`.

In [ ]:
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")

## Preprocessing Step

The pre-processing function uses scikit-learn StandardScaler to scale the features and convert them to NumPy.

Note that `keep_alive_period_in_seconds` parameter in @step decorator indicates how many seconds we want to keep the instance alive, waiting to be reused for the next pipeline step execution. Setting this parameter speeds up the pipeline execution because we reduce the launching of new instances to execute pipeline steps. Note also that we override the default 1instance type with the parameter `instance_type` defined in the function parameters.  

This step returns the normalized/scaled training and test datasets as `NumPy` arrays to be used in the next training and in evaluation stpes. 

In [ ]:
@step(
    name="data-preprocessing",
    instance_type=instance_type,
    keep_alive_period_in_seconds=600,
)
def preprocess(x_train, x_test, y_train, y_test):
    from sklearn.preprocessing import StandardScaler
    
    scaler = StandardScaler()
    scaler.fit(x_train.to_numpy())
    x_train_transformed_npy = scaler.transform(x_train.to_numpy())
    print(f"x_train_transformed_npy: {x_train_transformed_npy}")
    x_test_transformed_npy = scaler.transform(x_test.to_numpy())
    print(f"x_test_transformed_npy: {x_test_transformed_npy}")
    y_train_transformed_npy = y_train.to_numpy()
    print(f"y_train_transformed_npy: {y_train_transformed_npy}")
    y_test_transformed_npy = y_test.to_numpy() 
    print(f"y_test_transformed_npy: {y_test_transformed_npy}")
    
    return(x_train_transformed_npy, x_test_transformed_npy, y_train_transformed_npy, y_test_transformed_npy)

## Training Step

We train a TensorFlow model in this training step, using @step-decorated function with the normalized/scaled California housing training and test datasets as `NumPy` arrays. Both training and test datasets are coming from the output of the previous pre-processing step. Note also that we override the default 1instance type with the parameter `instance_type` defined in the function parameters. 

This step returns the TensorFlow model to be used in the next evaluation step. 

In [ ]:
import tensorflow as tf
    
def get_model():
    inputs = tf.keras.Input(shape=(8,))
    hidden_1 = tf.keras.layers.Dense(8, activation='tanh')(inputs)
    hidden_2 = tf.keras.layers.Dense(4, activation='sigmoid')(hidden_1)
    outputs = tf.keras.layers.Dense(1)(hidden_2)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

@step(
    name="model-training",
    instance_type=instance_type,
    keep_alive_period_in_seconds=600,
)
def train(x_train, x_test, y_train, y_test):      
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    
    if tf.config.list_physical_devices('GPU'):
        device = '/GPU:0'
    else:
        device = '/CPU:0'
    print(f"will use: {device}")
    
    batch_size = 128
    epochs = 25
    learning_rate = 0.01
    print('batch_size = {}, epochs = {}, learning rate = {}'.format(batch_size, epochs, learning_rate))

    with tf.device(device):
        model = get_model()
        optimizer = tf.keras.optimizers.SGD(learning_rate)
        model.compile(optimizer=optimizer, loss='mse')
        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                  validation_data=(x_test, y_test))

        # evaluate on test set
        scores = model.evaluate(x_test, y_test, batch_size, verbose=2)
        print("\nTest MSE :", scores)
        return(model)

## Evaluation Step

In this step, we create a @step-decorated function evaluate the trained TensorFlow model on the test dataset. Note also that we override the default 1instance type with the parameter `instance_type` defined in the function parameters. 

This step returns a report dictionary containing the `MSE` score. 

In [ ]:
evaluation_step_name = "model-evaluation"

@step(
    name=evaluation_step_name,
    instance_type=instance_type,
    keep_alive_period_in_seconds=600,
)
def evaluate(model, x_test, y_test):
    scores = model.evaluate(x_test, y_test, verbose=2)
    print("\nTest MSE :", scores)
    report_dict = {"mse": str(scores)}
    print(f"report_dict: {report_dict}")
    return(report_dict)

## Putting everything together: creating the Pipeline and running the pipeline execution

We connect all defined pipeline `@step` functions into a multi-step pipeline. Then, we submit and execute the pipeline.

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

step_process_result = preprocess(x_train, x_test, y_train, y_test)
step_train_result = train(step_process_result[0], step_process_result[1], step_process_result[2], step_process_result[3])
step_evaluation_result = evaluate(step_train_result, step_process_result[1], step_process_result[3]) 


pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        instance_type,
    ],
    steps=[
        step_evaluation_result,
    ],
)

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
execution = pipeline.start()

In [ ]:
execution.describe()

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()

## Getting the result of the evaluation step

We will retrieve the output of the evaluation step, which is the report dictionary containing the `MSE` score. 

In [ ]:
report_dict = execution.result(step_name=evaluation_step_name)
print(report_dict)

## Clean up Resources

### Delete pipeline

In [ ]:
pipeline.delete()